In [6]:
from keras.applications.nasnet import NASNetMobile as NASNet
from keras.applications.xception import Xception
from keras.applications.mobilenetv2 import MobileNetV2

from keras.utils import multi_gpu_model
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.python.client import device_lib
import os
import numpy as np
import tensorflow as tf
from keras import backend as K


In [2]:
K.set_learning_phase(0)


TRAIN_PATH = '../../data/img_data/small/recognized'
# TEST_PATH = '../../data/img_data/recognized'

# batch size used for validation and training
BATCH_SIZE = 16

# Number of passes over the data
TRAIN_EPOCHS = 5

# Percent of the data used for validation
VAL_SPLIT = 0.2

# size of our images
IMG_SIZE = (256, 256)

# determines the number of classes and total # of samples
walk_dir = os.walk(TRAIN_PATH)

# NUM_CLASSES = len(list(walk_dir)[0][1])
NUM_CLASSES = 340
NUM_SAMPLES = 36410339
# for root, dirs, files in walk_dir:
#     NUM_SAMPLES += len(files)

In [3]:
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

In [11]:
# define the model
# input_tensor = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 1))
# input_tensor = Conv2D(filters=3, kernel_size=5, strides=1)(input_tensor)
# model = NASNet(input_shape=(256, 256, 3), include_top=False, classes=NUM_CLASSES)

# model = Sequential()
# model.add(Conv2D(128, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(NUM_CLASSES, activation='softmax'))

single_threaded_model = MobileNetV2(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 1), alpha=1., weights=None, classes=NUM_CLASSES)
model = multi_gpu_model(single_threaded_model, gpus=4)
model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

ValueError: To call `multi_gpu_model` with `gpus=4`, we expect the following devices to be available: ['/cpu:0', '/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3']. However this machine only has: ['/cpu:0']. Try reducing `gpus`.

In [5]:
# make transformers
train_datagen = ImageDataGenerator(
#         rotation_range=30,
#         shear_range=0.2,
#         zoom_range=[0.9, 1],
#         horizontal_flip=True,
#         fill_mode='nearest',
        validation_split=VAL_SPLIT)

#preprocessing_function = lambda x: np.asarray(np.dstack((x, x, x)), dtype=np.uint8)

In [6]:
# make generators
train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=IMG_SIZE,
        color_mode="grayscale",
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        interpolation='nearest',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=IMG_SIZE,
        color_mode="grayscale",
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        interpolation='nearest',
        subset='validation')

Found 36483 images belonging to 340 classes.
Found 8960 images belonging to 340 classes.


In [7]:
# define callbacks
callbacks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1)
]

In [8]:
# fit the model
history = model.fit_generator(
            train_generator,
            steps_per_epoch=((1-VAL_SPLIT)*NUM_SAMPLES)//BATCH_SIZE,
            epochs=TRAIN_EPOCHS,
            validation_data=validation_generator,
            validation_steps=(VAL_SPLIT*NUM_SAMPLES)//BATCH_SIZE,
            callbacks = callbacks)

Epoch 1/5
     23/1820516 [..............................] - ETA: 1404:52:07 - loss: nan - categorical_crossentropy: nan - categorical_accuracy: 0.0082 - top_3_accuracy: 0.0435

KeyboardInterrupt: 

In [ ]:
# # Save model weights
model.save_weights('test.h5')

with open('acc.txt', 'w') as f:
    f.write(history.history['acc'])

with open('loss.txt', 'w') as f:
    f.write(history.history['loss'])

with open('val_acc.txt', 'w') as f:
    f.write(history.history['val_acc'])

with open('val_loss.txt', 'w') as f:
    f.write(history.history['val_loss'])
